### 12/02/2021:  the vstart corpus is captured by going ot each of the 37 pages in vstart and capturing:
<ul>
    <li>Page:  URL for the page</li>
    <li>Title:  Title of page</li>
    <li>What:  "What" section on the page</li>
    <li>Why:  "Why" section on the page</li>
</ul>

In [1]:
import sys
print (sys.version)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


In [2]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

corpusFileName is the input file with lines that give context of the lines that follow: <br/>
\>>>Page:  page url <br/>
\>>>Title:  page title <br/>
\>>>What:  text of "What" on the page <br/>
\>>>Why:  text of "Why" on the pgae<br/>
<br/>
ldaLocation is the output of the LDA process, from 3 ODA groups through 9 LDA groups

In [3]:
corpusFileName = "C:\\Users\\mlaux\\OneDrive - Ventera Corporation\\Documents\\02.Projects\\04.HAIV_vstart\\002.WebCapture\\vstartCorpus_20211202.txt"
corpusCsvLocation = "C:\\Users\\mlaux\\OneDrive - Ventera Corporation\\Documents\\02.Projects\\04.HAIV_vstart\\002.WebCapture\\vstartCorpus.csv"
ldaLocation = "C:\\Users\\mlaux\\OneDrive - Ventera Corporation\\Documents\\02.Projects\\04.HAIV_vstart\\003.LDA\\titleLDA.csv"
termFreqsLocation = "C:\\Users\\mlaux\\OneDrive - Ventera Corporation\\Documents\\02.Projects\\04.HAIV_vstart\\003.LDA\\termFreqs.csv"

In [4]:
stopWords = [
    "a",
"about",
"actually",
"almost",
"also",
"although",
"always",
"am",
"an",
"and",
"any",
"are",
"as",
"at",
"be",
"became",
"become",
"but",
"by",
"can",
"could",
"did",
"do",
"does",
"each",
"either",
"else",
"for",
"from",
"had",
"has",
"have",
"hence",
"how",
"i",
"if",
"in",
"is",
"it",
"its",
"just",
"may",
"maybe",
"me",
"might",
"mine",
"must",
"my",
"mine",
"must",
"my",
"neither",
"nor",
"not",
"of",
"oh",
"ok",
"when",
"where",
"whereas",
"wherever",
"whenever",
"whether",
"which",
"while",
"who",
"whom",
"whoever",
"whose",
"why",
"will",
"with",
"within",
"without",
"would",
"yes",
"yet",
"you",
"your"
]

In [7]:
pages = []
titles = []
whats = []
whys = []

inx = 0

with open(corpusFileName, encoding="utf8") as f:
    lines = f.readlines()

currentCommand = ""    
    
for line in lines:
    ls = line.strip()
    if ls == ">>>Page:":
        if inx > 0:
            pages.append(page)
            titles.append(title)
            whats.append(what)
            whys.append(why)
        currentCommand = ls
        inx = inx + 1
        page = ""
        title = ""
        what = ""
        why = ""
    elif ls == ">>>Title:":
        currentCommand = ls
    elif ls == ">>>What:":
        currentCommand = ls
    elif ls == ">>>Why:":
        currentCommand = ls
    else:
        if currentCommand == ">>>Page:":
            page = ls
        elif currentCommand == ">>>Title:":
            title = title + " " + ls
        elif currentCommand == ">>>Why:":
            why = why + " " + ls
        elif currentCommand == ">>>What:":
            what = what + " " + ls

if inx > 0:
    pages.append(page)
    titles.append(title)
    whats.append(what)
    whys.append(why)
            
corpusDict = {'page': pages, 'title': titles, 'what': whats, 'why': whys}
dfCorpus = pd.DataFrame(data=corpusDict)
dfCorpus.to_csv(corpusCsvLocation, \
                  sep = "|", index_label = "index_label")

In [8]:

ldaFitted = {}

dfCorpus = pd.read_csv(corpusCsvLocation, sep = "|")
corpus = [f"{row['title']} {row['what']} {row['why']}" for inx, row in dfCorpus.iterrows()]
vectorizer = CountVectorizer(stop_words = stopWords)
X = vectorizer.fit_transform(corpus)
for inx in range(3, 10):
    lda = LatentDirichletAllocation(n_components=inx, random_state = 0)
    lda.fit(X)
    a = lda.transform(X)
    ldaFitted[inx] = a

print(vectorizer.get_stop_words())    
print(vectorizer.get_feature_names())
print(X.toarray())

vecNames = vectorizer.get_feature_names()
dictFreqs = {vecNames[i]: X.toarray()[:, i] for i in range(len(vecNames))}
dictFreqs
datasetFreqs = pd.DataFrame(dictFreqs)
datasetFreqs.to_csv(termFreqsLocation, index_label = "index_label")
ldaFitted    

frozenset({'within', 'yet', 'an', 'also', 'nor', 'had', 'can', 'neither', 'but', 'be', 'when', 'without', 'yes', 'you', 'its', 'are', 'about', 'am', 'almost', 'it', 'from', 'any', 'if', 'whether', 'do', 'i', 'your', 'might', 'became', 'mine', 'does', 'could', 'whose', 'which', 'not', 'by', 'maybe', 'how', 'will', 'oh', 'in', 'wherever', 'a', 'has', 'whom', 'for', 'and', 'just', 'must', 'my', 'while', 'always', 'why', 'would', 'whoever', 'hence', 'as', 'where', 'whenever', 'is', 'did', 'actually', 'whereas', 'with', 'become', 'ok', 'who', 'either', 'else', 'although', 'each', 'of', 'have', 'at', 'may', 'me'})
['accelerates', 'acceptance', 'accepted', 'accepting', 'accessible', 'accomplish', 'according', 'accountable', 'accurate', 'achieve', 'acquire', 'across', 'act', 'action', 'actionable', 'actions', 'activities', 'activity', 'actual', 'adaptive', 'add', 'added', 'addition', 'additionally', 'address', 'adds', 'adjusts', 'administration', 'adoption', 'affinity', 'after', 'against', 'ag

{3: array([[0.00582353, 0.98775174, 0.00642473],
        [0.00166164, 0.0017509 , 0.99658747],
        [0.00468939, 0.99004398, 0.00526663],
        [0.0020025 , 0.00228715, 0.99571035],
        [0.00341061, 0.99222712, 0.00436227],
        [0.00285767, 0.0031141 , 0.99402823],
        [0.0033107 , 0.00346411, 0.99322519],
        [0.00459107, 0.00509498, 0.99031395],
        [0.00293728, 0.99376626, 0.00329646],
        [0.00219274, 0.00281046, 0.9949968 ],
        [0.00439734, 0.00458338, 0.99101928],
        [0.00237039, 0.99505407, 0.00257554],
        [0.00209396, 0.99559992, 0.00230612],
        [0.0029717 , 0.00327971, 0.99374859],
        [0.00395959, 0.00424728, 0.99179313],
        [0.00213405, 0.0023754 , 0.99549055],
        [0.00222546, 0.00231567, 0.99545887],
        [0.00230366, 0.00248888, 0.99520746],
        [0.00255219, 0.00281993, 0.99462788],
        [0.00568037, 0.00596682, 0.98835281],
        [0.00485319, 0.00557609, 0.98957071],
        [0.00306986, 0.00322478

In [9]:
a2 = []
for k in ldaFitted:
    for i in range(len(ldaFitted[k])):
        a2.append({**{"n": k, "row": i, "title": titles[i].strip()} , **{ \
            str(j): ldaFitted[k][i][j] if j < len(ldaFitted[k][i]) else 0.0
            for j in range(10)}})

# {**orig, **extra}        

In [10]:
pd.DataFrame(a2).to_csv(ldaLocation, index_label = "index_label")
a2

[{'n': 3,
  'row': 0,
  'title': 'Contextual Inquiry',
  '0': 0.005823534075829886,
  '1': 0.9877517383746245,
  '2': 0.006424727549545661,
  '3': 0.0,
  '4': 0.0,
  '5': 0.0,
  '6': 0.0,
  '7': 0.0,
  '8': 0.0,
  '9': 0.0},
 {'n': 3,
  'row': 1,
  'title': 'Design Kit',
  '0': 0.001661638135606275,
  '1': 0.0017508960492742369,
  '2': 0.9965874658151194,
  '3': 0.0,
  '4': 0.0,
  '5': 0.0,
  '6': 0.0,
  '7': 0.0,
  '8': 0.0,
  '9': 0.0},
 {'n': 3,
  'row': 2,
  'title': 'Ideation and Brainstorming',
  '0': 0.00468939022796668,
  '1': 0.9900439845078174,
  '2': 0.005266625264215933,
  '3': 0.0,
  '4': 0.0,
  '5': 0.0,
  '6': 0.0,
  '7': 0.0,
  '8': 0.0,
  '9': 0.0},
 {'n': 3,
  'row': 3,
  'title': 'Minimum Viable Product',
  '0': 0.002002503956519265,
  '1': 0.002287149796947602,
  '2': 0.9957103462465331,
  '3': 0.0,
  '4': 0.0,
  '5': 0.0,
  '6': 0.0,
  '7': 0.0,
  '8': 0.0,
  '9': 0.0},
 {'n': 3,
  'row': 4,
  'title': 'Prioritization',
  '0': 0.003410613922799098,
  '1': 0.9922271